In [5]:
%matplotlib qt
import pyxdf
import mne
import numpy as np
from mne.datasets import misc
import os
import matplotlib.pyplot as plt
import matplotlib
import time
print(os.getcwd())

fnames = ["C:/Users/elpid/OneDrive/Υπολογιστής/shielded_27_10_23/elpida/sub-elpida_ses-27_10_23_task-blinking_acq-shielded_hugin04_dots_run-001_eeg.xdf"]
# sub-LeviGN_ses-S001_task-DefaultEyes_acq-huginn4_run-001_eeg
raws = []
dataTimestamps = 0
for fnameIndex in range(len(fnames)): 
    streams, header = pyxdf.load_xdf(fnames[fnameIndex])
    # get sampling rate
    sfreq = float(streams[1]["info"]["nominal_srate"][0])

    # get data
    data = streams[1]["time_series"].T
    dataTimestamps = streams[1]["time_stamps"].T
    # get markers
    markers = streams[0]["time_series"].T
    markerTimestamps = streams[0]["time_stamps"].T

    #in case of there is a switch between stim channel and data channels
    if  data.shape[0] != 8:
        data = streams[0]["time_series"].T
        dataTimestamps = streams[0]["time_stamps"].T
        sfreq = float(streams[0]["info"]["nominal_srate"][0])
        markers = streams[1]["time_series"].T
        markerTimestamps = streams[1]["time_stamps"].T
        assert data.shape[0] == 8,  fnames[fnameIndex] + " has different channel amounts" # check that we have what we think we have (8 eeg channels)

    

    # get together the event channel with the data, make a "stim 014" type channel as annotation: time, duration, event type 
    # add markers: What needs to be done: we have markers as several events, and their timestamp. then, we have the data
    # and their timestamps, to the raw object we should add one more channel, initially full of zeros with the length of the measurement
    
    
    # This is because microvolts are being measured, but mne plot expects values in volts.
    markers = markers[0]
    data *= 1e-6 # uV -> V and preamp gain. IDK what will preamp gain do. (multiply / divide by gain? 24?)

    markerArr = np.zeros_like(data[0])
    
    for markerid in range(len(markers)):
        absolute_differences = np.abs(dataTimestamps - markerTimestamps[markerid])
        closest_index = np.argmin(absolute_differences)
        markerArr[closest_index] = markers[markerid]

    # TEST: this gives the amount of events, and also their index. divided by the sampling freq, you should see the time differences of
    #each part of the experiment. 
    nonzero_indices = np.nonzero(markerArr)
    num_nonzero = len(nonzero_indices[0])
    print("uniques", np.unique(markerArr))
    print("Number of nonzero elements:", num_nonzero)
    print("Indices of nonzero elements:", nonzero_indices[0] / sfreq)

    #add marker arr to data
    combined = np.c_[data.T,markerArr].T

    # create info object set channel names
    info = mne.create_info(["T6", "T4", "F8", "C4", "C3", "T5", "T3", "F7", "stim"], sfreq, ["eeg", "eeg","eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "stim"])
    raw = mne.io.RawArray(combined, info)
    raw.set_montage("standard_1020")

    #preliminary data analyzis: (fancy term for plotting it and looking at it). Task: review data and select a consistently bad channel for exclusion
    print("Task: review data and select a consistently bad channel for exclusion.")
    
    raw.plot(title=fnames[fnameIndex] + ", Task: review data and select consistently bad channels", duration=1)
    raw.compute_psd().plot()
    plt.title(fnames[fnameIndex])
    plt.show()
    raws.append((raw, fnames[fnameIndex]))

C:\Users\elpid\Downloads\thesis\ThesisRepository\scripts
uniques [0. 1.]
Number of nonzero elements: 5
Indices of nonzero elements: [ 11.412 132.532 253.692 374.844 495.984]
Creating RawArray with float64 data, n_channels=9, n_times=155564
    Range : 0 ... 155563 =      0.000 ...   622.252 secs
Ready.
Task: review data and select a consistently bad channel for exclusion.
Effective window size : 1.024 (s)


C:\Users\elpid\AppData\Local\Temp\ipykernel_7264\2253892762.py:75: UserWarning: Zero value in spectrum for channels C4, F7
  raw.compute_psd().plot()
C:\Users\elpid\AppData\Local\Temp\ipykernel_7264\2253892762.py:75: UserWarning: Infinite value in PSD for channels C4, F7.
These channels might be dead.
  raw.compute_psd().plot()


Channels marked as bad:
['F7', 'C4']


In [6]:
epochs = []
for rawpack in raws: 
    raw = rawpack[0]
    fname = rawpack[1]
    low_cut = 0.1
    high_cut = 50
    filtered_data = raw.copy().filter(low_cut,high_cut,method='iir')
    notch_freqs = [50, 100]
    filtered_data = filtered_data.notch_filter(freqs=notch_freqs, filter_length='auto', method='fir', notch_widths=4, picks="eeg")

    #get events    
    events = mne.find_events(filtered_data, stim_channel="stim")
    #events: 1: start 
    #open close experiment: event at 0s, open at 0 - 60s, closed at 60 - 120s
    #setup values above which you reject (peak to peak)
    reject_criteria = dict(eeg=200e-6)

    #set flat criteria (epochs where nothing happens really (min peak to peak))
    flat_criteria = dict(eeg=1e-6)

    #make epochs: eyes open
    stimEpochs = mne.Epochs(filtered_data, events, event_id=1, tmin=0, tmax=60, baseline=(0, 0), preload=True, reject=reject_criteria, flat=flat_criteria)

    #stimEpochs.plot_drop_log()
    #stimEpochs.plot(title=fname+"filtered")
    #epochs.append((stimEpochs, fnames[fnameIndex]))
    stimEpochs.compute_psd().plot()
    plt.title("Open " + fname)
    plt.show()

    stimEpochs = mne.Epochs(filtered_data, events, event_id=1, tmin=60, tmax=120, baseline=(60, 60), preload=True, reject=reject_criteria, flat=flat_criteria)
    stimEpochs.compute_psd().plot()
    plt.title("Closed " + fname)
    plt.show()
    
    
    

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.10, 50.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 sec)

5 events found
Event IDs: [1]
Not setting metadata
5 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 5 events and 15001 original time points ...
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Ave

In [7]:
from datetime import datetime, timedelta

def count_occasions_outside_timeframe(timestamps, timeframe_seconds):
    # Initialize the counter to zero
    count = 0
    
    # Convert timeframe_seconds to a timedelta object for comparison
    #timeframe = timedelta(seconds=timeframe_seconds)
    
    # Iterate through the array up to the second-to-last timestamp
    for i in range(len(timestamps) - 1):
        current_timestamp = timestamps[i]
        next_timestamp = timestamps[i + 1]
        
        # Calculate the time difference between current and next timestamps
        time_diff = next_timestamp - current_timestamp
        
        # Compare with the specified timeframe
        if time_diff < timeframe_seconds:
            count += 1
    
    return count

num = count_occasions_outside_timeframe(dataTimestamps, 0.003999)
print("number: ", num, "ratio: ", num/len(dataTimestamps))

number:  155563 ratio:  0.9999935717775321


In [8]:
np.diff(dataTimestamps).mean()
import matplotlib.pyplot as plt
plt.plot(np.arange(0,len(dataTimestamps)-1,1),np.diff(dataTimestamps))
plt.show()


In [9]:
np.diff(dataTimestamps)

array([0.00399519, 0.00399519, 0.00399519, ..., 0.00399519, 0.00399519,
       0.00399519])